In [11]:
import numpy as np
import pandas as pd
import os
from pathlib import Path
import duckdb

base_dir = Path('c:\\Users\\henry\\OneDrive\\Personal Career\\Personal Projects\\GitHub\\Revenue-Sustainability-Analysis')
data_dir = Path(base_dir / 'Dataset')

## Define adopters vs. non adopters


In [12]:
os.chdir(data_dir)

df = pd.read_parquet('feature_usage.parquet')

# Remove the random ...1 introduced
# df = df.drop(columns='...1')
# df.to_parquet(data_dir / 'feature_usage.parquet')

In [13]:
os.chdir(base_dir / 'raw_data')

df = pd.read_csv('feature_usage.csv')

In [14]:
feats = pd.read_parquet(data_dir / 'feature_usage.parquet')

In [15]:
# pd.set_option('display.max_rows',None)

In [16]:
users = feats[feats['feature_name'] == 'feature_newai']

In [18]:
users.groupby("subscription_id")["usage_date"].nunique().reset_index(name="distinct_usage_days") \
    .sort_values(by=['distinct_usage_days', 'subscription_id'],
    ascending=[False,True])

,subscription_id,distinct_usage_days
25,S-0f2e98,3
35,S-148cf4,3
43,S-1a2a77,3
49,S-1cafc8,3
57,S-1fc27b,3
...,...,...
321,S-bb9f23,0
330,S-be509c,0
363,S-d11408,0
366,S-d3efa2,0


In [21]:
users.columns

Index(['usage_id', 'subscription_id', 'usage_date', 'feature_name',
       'usage_count', 'usage_duration_secs', 'error_count', 'is_beta_feature',
       'post_release', 'usage_pk', 'synthetic', '_usage_day', 'usage_day'],
      dtype='object')

In [27]:
users[users['subscription_id'] == 'S-0ce88b']

,usage_id,subscription_id,usage_date,feature_name,usage_count,usage_duration_secs,error_count,is_beta_feature,post_release,usage_pk,synthetic,_usage_day,usage_day
43399,U-sage43399,S-0ce88b,<NA>,feature_newai,9,3170,0,True,True,None,FALSE,None,2024-01-27


In [22]:
distinct_days = users.groupby('subscription_id')['usage_date'].nunique().reset_index(name='distinct_usage_days')

In [26]:
distinct_days[distinct_days['distinct_usage_days'] == 0]

,subscription_id,distinct_usage_days
20,S-0ce88b,0
28,S-10ab8d,0
41,S-1862f2,0
50,S-1cff18,0
53,S-1eb467,0
79,S-2e1bc6,0
99,S-396a84,0
129,S-4a7821,0
140,S-51dd23,0
143,S-56122a,0


In [23]:
distinct_days.groupby('distinct_usage_days')['subscription_id'].nunique().reset_index(name='num_users').sort_values('distinct_usage_days')

,distinct_usage_days,num_users
0,0,25
1,1,306
2,2,77
3,3,18
